## [Python 使用 Google 雲端硬碟 API 自動進行文字辨識教學](https://blog.gtwang.org/programming/automation-of-google-ocr-using-python-tutorial/)


開發程式之前，請先參考[Google Developers 官方文件](https://developers.google.com/drive/v3/web/quickstart/python)的 Python 範例，看懂之後再把程式碼改成自動上傳圖片、下載文字檔的 Python 程式，以下是完整的程式碼。

In [4]:
!pip3 install --upgrade google-api-python-client gspread oauth2client

Requirement already up-to-date: google-api-python-client in c:\users\jiangsir\anaconda3\lib\site-packages (1.7.4)


You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
#!/usr/bin/python3
from __future__ import print_function
import httplib2
import os
import io

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
from apiclient.http import MediaFileUpload, MediaIoBaseDownload

try:
  import argparse
  flags = argparse.ArgumentParser(parents=[tools.argparser]).parse_args()
except ImportError:
  flags = None

SCOPES = 'https://www.googleapis.com/auth/drive'
CLIENT_SECRET_FILE = 'client_id.json'
APPLICATION_NAME = 'Python OCR'

def get_credentials():
  """取得有效的憑證
     若沒有憑證，或是已儲存的憑證無效，就會自動取得新憑證

     傳回值：取得的憑證
  """
  credential_path = os.path.join("./", 'google-ocr-credential.json')
  store = Storage(credential_path)
  credentials = store.get()
  if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
      credentials = tools.run_flow(flow, store, flags)
    else: # Needed only for compatibility with Python 2.6
      credentials = tools.run(flow, store)
    print('憑證儲存於：' + credential_path)
  return credentials

def main():

  # 取得憑證、認證、建立 Google 雲端硬碟 API 服務物件
  credentials = get_credentials()
  http = credentials.authorize(httplib2.Http())
  service = discovery.build('drive', 'v3', http=http)

  # 包含文字內容的圖片檔案（png、jpg、bmp、gif、pdf）
  imgfile = 'sample.jpg'

  # 輸出辨識結果的文字檔案
  txtfile = 'output.txt'

  # 上傳成 Google 文件檔，讓 Google 雲端硬碟自動辨識文字
  mime = 'application/vnd.google-apps.document'
  res = service.files().create(
    body={
      'name': imgfile,
      'mimeType': mime
    },
    media_body=MediaFileUpload(imgfile, mimetype=mime, resumable=True)
  ).execute()

  # 下載辨識結果，儲存為文字檔案
  downloader = MediaIoBaseDownload(
    io.FileIO(txtfile, 'wb'),
    service.files().export_media(fileId=res['id'], mimeType="text/plain")
  )
  done = False
  while done is False:
    status, done = downloader.next_chunk()

  # 刪除剛剛上傳的 Google 文件檔案
  service.files().delete(fileId=res['id']).execute()

if __name__ == '__main__':
  main()

usage: ipykernel_launcher.py [-h] [--auth_host_name AUTH_HOST_NAME]
                             [--noauth_local_webserver]
                             [--auth_host_port [AUTH_HOST_PORT [AUTH_HOST_PORT ...]]]
                             [--logging_level {DEBUG,INFO,WARNING,ERROR,CRITICAL}]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\jiangsir\AppData\Roaming\jupyter\runtime\kernel-7840ac8b-42f3-4a59-91c0-dcab5903ece3.json


SystemExit: 2

C:\Users\jiangsir\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## [使用Python上傳資料到Google試算表](https://sites.google.com/site/zsgititit/home/python-cheng-shi-she-ji/shi-yongpython-shang-chuan-zi-liao-daogoogle-shi-suan-biao)

In [10]:
import sys
import time
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials as SAC
GDriveJSON = 'client_id.json'
GSpreadSheet = 'UploadByPython'
WaitSecond = 60
print('將資料記錄在試算表' ,GSpreadSheet , '每' ,WaitSecond ,'秒')
print('按下 Ctrl-C中斷執行')
count = 1
while True:
    try:
        scope = ['https://spreadsheets.google.com/feeds']
        key = SAC.from_json_keyfile_name(GDriveJSON, scope)
        gc = gspread.authorize(key)
        worksheet = gc.open(GSpreadSheet).sheet1
    except Exception as ex:
        print('無法連線Google試算表 ', ex)
        sys.exit(1)
    worksheet.append_row((datetime.datetime.now(), count))
    count = count+1
    print('新增一列資料到試算表' ,GSpreadSheet)
    time.sleep(WaitSecond)

將資料記錄在試算表 UploadByPython 每 60 秒
按下 Ctrl-C中斷執行
無法連線Google試算表  ('Unexpected credentials type', None, 'Expected', 'service_account')


SystemExit: 1

C:\Users\jiangsir\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
